## Radiomics

La radiómica es un campo de estudio médico que busca extraer características cuantitativas de imágenes médicas, convirtiendo las imágenes en datos minables y aplicando el análisis de estos datos para brindar soporte a la decisión clínica. De la misma forma que la genómica o la proteómica, este campo de la medicina de precisión busca encontrar patrones explicativos en sets de gran cantidad de datos. Se contrapone a la práctica tradicional de tratar las imágenes médicas como meras imágenes demostrativas destinadas únicamente a interpretación visual. La radiómica contiene estadística de primer, segundo y alto orden, que combinados con datos clínicos del paciente pueden generar modelos que potencialmente podrían mejorar la precisión diagnóstica, prognóstico y predictiva. 

La radiómica ha demostrado un buen desempeño en otros tipos de cáncer, especialmente de cerebro y de próstata, ya que son patologías de alta prevalencia y para las que se cuenta con sets de imágenes de gran tamaño. El pipeline de radiomics consiste en la adquisición de imagen, la segmentación de la región de interés, un posible preprocesamiento de la imagen (filtrado, resampleo, padding, cropping, entr otros) y finalmente la extracción de los "radiomic features". Para Python existe el paquete [pyradiomics](https://pyradiomics.readthedocs.io/en/latest/customization.html) que explica cómo configurar este proceso en su documentación.
 
<img src="radiomics.jpg" alt="Drawing" style="width:40%;"/>


## Dataset de Gliomas 

El dataset [TCGA-LGG](https://wiki.cancerimagingarchive.net/display/Public/TCGA-LGG#49dd0de7af2043e680c4ecb36fe605a3) es un dataset público que combina información genética del Cancer Genome Archive (TCGA) con imágenes de resonancia magnética de cerebro del Cancer Imaging Archive (TCIA) , pertenecientes a pacientes diagnosticados con glioma de bajo grado (Low Grade Glioma). Este tipo de tumor puede estar asociado a una mutación genética, en cuyo caso el tratamiento será diferente. Es de interés poder predecir esta mutación ya que el análisis genómico es muy costoso.

In [1]:
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/rn-2019-itba/Clase-7--Regularizacion/master/radiomics_gliomas.csv')

In [2]:
data.head()

,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,original_firstorder_Mean,original_firstorder_MeanAbsoluteDeviation,original_firstorder_Median,...,wavelet-LLH_firstorder_Uniformity,wavelet-LLH_firstorder_Variance,wavelet-LLH_glcm_Autocorrelation,wavelet-LLH_glcm_ClusterProminence,wavelet-LLH_glcm_ClusterShade,wavelet-LLH_glcm_ClusterTendency,wavelet-LLH_glcm_Contrast,wavelet-LLH_glcm_Correlation,wavelet-LLH_glcm_DifferenceAverage,Mutacion
0,155.615205,353.184777,2.878215e+08,5.809761,97.306704,2.834441,434.351234,266.625929,60.124082,277.726650,...,0.041838,1257.667670,644.310749,58710.668931,-51.434398,128.596721,69.240755,0.301791,6.214115,1
1,191.101389,372.599007,2.066994e+09,5.637016,99.693104,2.549367,424.250372,293.920752,55.622405,306.916096,...,0.110662,212.043507,162.874178,3080.122135,-96.878003,27.905486,5.517043,0.668519,1.679473,1
2,162.543544,413.165807,1.496800e+08,5.860199,103.001719,2.833426,475.078627,261.387435,70.544241,246.738413,...,0.045860,1237.007712,492.419360,69561.725366,-959.722953,162.728062,60.573934,0.468938,5.976991,1
3,247.658193,420.426441,2.117501e+08,5.607336,113.453612,2.119373,474.868965,335.706343,58.695066,337.126368,...,0.072517,613.130293,696.942500,15338.156162,-432.844674,52.406665,21.422252,0.386653,3.373647,1
4,191.321605,404.324298,5.687897e+08,5.871768,96.504857,3.300441,476.342717,314.116180,62.270515,328.312934,...,0.057828,724.368720,857.655164,24572.757630,-132.585964,73.449313,50.763610,0.190613,5.220063,0


In [3]:
X = data[data.columns[:-2]].values
print(X.shape)
         
Y = data['Mutacion'].values
print(Y.shape)

(150, 640)
(150,)


Dado que tenemos pocas observaciones para la cantidad de variables, vamos a aplicar cross validation para hacer el 'tuning' de hiperparámetros al entrenar. La metodología es la siguiente: por cada variante de modelo que deseamos evaluar, hacemos un 'fit' para todos los 'folds' menos uno (folds de entrenamiento), y un 'predict' para el fold restante (fold de validación). Repetimos esto para todas las combinaciones de folds disponibles. Finalmente, consideramos el desempeño de validación de ese modelo como el promedio de los scores obtenidos en todas nuestras corridas de fit-predict. 

Por ejemplo: queremos comparar un ModeloA (con una cierta combinación de hiperparámetros), con el ModeloB (otra combinación diferente), y haciendo un cross-validation de 3 folds, es decir, dividiendo nuestro dataset en tres: fold1, fold2, fold3.

1) Nuevo ModeloA. Fit de ModeloA con fold1+fold2

2) Predict de ModeloA ajustado en (1) sobre fold3 --> "score1"

3) Nuevo ModeloA. Fit de ModeloA con fold1+fold3

4) Predict de ModeloA ajustado en (3) sobre fold2 --> "score2"

5) Nuevo ModeloA. Fit de ModeloA con fold2+fold3

6) Predict de ModeloA ajustado en (5) sobre fold1 --> "score3"

7) ScoreA será el promedio de score1, score2 y score3.

Repetimos los pasos para ModeloB, y obtenemos el ScoreB. La comparación entre ScoreA y ScoreB nos definirá qué combinación de hiperparámetros es mejor.

In [4]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(10) #10 folds
splited_indexs = skf.split(X, Y)

Visualizamos nuestros folds de cross-validation:

In [5]:
i=0
training_sets = []
for train_index, test_index in splited_indexs:
    i=i+1
    print("CV dataset:", i)
    print(train_index.shape, test_index.shape)
    dictionary = {'X_train':X[train_index], 'y_train':Y[train_index], 'X_test':X[test_index],'y_test':Y[test_index]}
    training_sets.append(dictionary)

CV dataset: 1
(134,) (16,)
CV dataset: 2
(134,) (16,)
CV dataset: 3
(134,) (16,)
CV dataset: 4
(134,) (16,)
CV dataset: 5
(135,) (15,)
CV dataset: 6
(135,) (15,)
CV dataset: 7
(136,) (14,)
CV dataset: 8
(136,) (14,)
CV dataset: 9
(136,) (14,)
CV dataset: 10
(136,) (14,)


El método cross_validate ejecuta el proceso descripto anteriormente. Debemos especificar el modelo que queremos ajustar ('estimator'), nuestros datos de entrada ('X'), nuestros targets ('y'), y un objeto de cross-validation ('cv'. Ver documentación, este objeto soporta también otros formatos)

Por ejemplo, comencemos evaluando un modelo Lasso con hiperparámetro lambda de 0.5 (Recordar que en sklearn este hiperpárametro es llamado alpha). Como score utilizaremos error cuadrático medio. (¿Por qué no podemos usar Accuracy?)


In [5]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_validate
import numpy as np
alfa = 0.5
clf_lasso_A = Lasso(alpha=alfa, max_iter=1e5)
cross_val_outputs_A = cross_validate(estimator=clf_lasso_A, X=X, y=np.array(Y), cv=skf, scoring='neg_mean_squared_error')
print('Error en cada fold:',  cross_val_outputs_A['test_score'])

C:\Users\UsuarioHI\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\UsuarioHI\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\UsuarioHI\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\UsuarioHI\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You migh

[-1.51491031e+03 -1.21446419e+00 -5.73364017e-01 -5.51777575e+01
 -1.94515317e-01 -6.29410546e+03 -2.74221938e+00 -1.45447285e+04
 -3.22863156e+02 -3.06213008e-01]


C:\Users\UsuarioHI\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [9]:
scoreA = np.mean(cross_val_outputs_A['test_score'])
print('Score A:', scoreA)
print('Error cuadratico medio A:', abs(scoreA))

Error A: -2273.681594647875


In [10]:
alfa = 0.1
clf_lasso_B = Lasso(alpha=alfa, max_iter=1e5)
cross_val_outputs_B = cross_validate(estimator=clf_lasso_B, X=X, y=np.array(Y), cv=skf, scoring='neg_mean_squared_error')
print('Error en cada fold:', cross_val_outputs_B['test_score'])

C:\Users\UsuarioHI\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\UsuarioHI\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\UsuarioHI\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\Users\UsuarioHI\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You migh

[-5.94996606e+03 -8.66179587e+00 -1.39161740e+00 -1.82763141e+01
 -1.63359687e-01 -6.70377671e+04 -2.27517543e+00 -5.85954353e+03
 -2.37786751e+00 -5.56209590e+02]


C:\Users\UsuarioHI\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [12]:
scoreB = np.mean(cross_val_outputs_B['test_score'])
print('Score B:', scoreB)
print('Error cuadratico medio B:', abs(scoreB))

Score B: -7943.663244697639
Error cuadratico medio B: 7943.663244697639


Vemos que en este caso obtenemos mejor Score con el primer modelo. (Igualmente parece un score muy malo)???

**Red neuronal**

Desarrollá una red neuronal para predecir si un glioma cerebral presenta o no mutación genética a partir de las características radiómicas de nuestro dataframe. Probá variar los distintos hiperparámetros propios de una red neuronal densa: 

- Cantidad de capas ocultas y cantidad de unidades por capa

- Capas con dropout y su dropout rate (p)

- Capas con Kernel_normalizer con L1 o L2

- 

In [14]:
from keras.models import Sequential
from keras.layers import Dense
lr=0.01
input_shape = X.shape[1]
hidden_units=32
output_size=2

model = Sequential()
sgd = optimizers.SGD(lr=lr)
model.add(Dense(hidden_units,input_dim=input_shape,  activation='relu'))
model.add(Dense(output_size, 
                activation='sigmoid', 
                kernel_initializer='zeros', 
                name='Salida'
               ))
model.compile(loss = 'binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

history = model.fit(X,Y, epochs=10)

ModuleNotFoundError: No module named 'keras'

In [13]:
print(X.shape)

(150, 640)
